In [ ]:
!nvidia-smi
!kill process_id

In [ ]:
!pip install transformers
!pip install simpletransformers
!pip install rouge_metric

In [ ]:
import logging
import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

import wandb
!wandb login 6fcfd33f292c19c9a44b565ee505839e5381b21e

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import shutil
dele = True

if dele:
  shutil.rmtree('/content/outputs')
  shutil.rmtree('/content/runs')
  shutil.rmtree('/content/cache_dir')
  shutil.rmtree('/content/wandb')


In [ ]:

df = pd.read_csv(r'/content/data.txt', sep = ';')
X = df["input_text"]
y = df['target_text']



In [ ]:
l = []

for i in (y):
  l.append(len(i))

import numpy as np
print(np.min(l))
print(np.median(l))
print(np.max(l))

241
1043.0
3054


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2020 )
train = pd.concat([X_train,y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)



In [ ]:
model_args = Seq2SeqArgs()

# Configure the model
model_args = Seq2SeqArgs()
model_args.num_train_epochs = 15

# misc hyperparams
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.overwrite_output_dir = True
model_args.save_eval_checkpoints = False
model_args.no_save =True
model_args.fp16 = False
#model_args.best_model_dir = '/content/'
model_args.wandb_project = "vt1-seq2seq"

# numeric hyperparams
model_args.max_seq_length = 128 # Maximum sequence length the model will support. docs/usge
model_args.max_length = 1024 # The max length of the sequence to be generated. Between 0 and infinity. Default to 20.
model_args.num_return_sequences = 1
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.evaluate_during_training_steps = 2500
model_args.learning_rate = 4e-5

model_args.sliding_window = True
#model_args.do_sample = False
#model_args.top_k = 50
#model_args.top_p = 0.95


# Training with sliding window



In [ ]:

# Configure the model
model_args = Seq2SeqArgs()

model_args.max_seq_length = 512 # Maximum sequence length the model will support. docs/usge
model_args.max_length = 3000 # The max length of the sequence to be generated. Between 0 and infinity. Default to 20.
model_args.min_length = 1000 # The max length of the sequence to be generated. Between 0 and infinity. Default to 20.

model_args.sliding_window = True


In [ ]:
# Initialize model
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name='facebook/bart-large-cnn' ,#"facebook/bart-base"
    args=model_args
    #use_cuda=False,
)

In [ ]:
# Train the model
model.train_model(train_data = train, show_running_loss = True, eval_data = test )
# Save model to wandb


In [ ]:
sample = [train['input_text'][0]]
sample

["Um I'm Craig and I'm User Interface. Yeah. Well, my favourite animal would be a monkey. Then they're small cute and furry, and uh when planet of the apes becomes real, I'm gonna be up there with them. Yeah. I know um My parents went out and bought um remote controls because um they got fed up of having four or five different remote controls for each things the house. So um for them it was just how many devices control. Uh.\nMm-hmm. Great. And I'm Andrew and I'm uh our marketing expert. Mm-hmm. Mm-hmm. Yeah, that's that's it. Yeah. I will go. That's fine. Alright. So This one here, right? Okay. Very nice. Alright. My favourite animal is like A beagle. Um charac favourite characteristics of it? Is that right? Uh, right, well basically um high priority for any animal for me is that they be willing to take a lot of physical affection from their family. And, yeah that they have lots of personality and uh be fit and in robust good health. So this is blue. Blue beagle. My family's beagle. I

In [ ]:
p = model.predict(sample)

In [ ]:
p

["Craig's favourite animal would be a monkey. Andrew's favourite is a beagle. The price of the game is twelve fifty. The cost of production is twelve hundred. The selling price is is that wholesale or retail? Like on the shelf. The game is on sale now."]

In [ ]:
import numpy as np
#np.save("weights", weights)
#wandb.save("weights.npy")

In [ ]:
import os
preds = []
preds = model.predict(test['input_text'].tolist())



In [ ]:

pred = pd.DataFrame({'preds':preds})
#test['target_text']
preds[0]

'The project manager opened the meeting by stating that the project manager had instructed the team to prepare a new project requirement. The team then discussed the project budget and the project requirements. The project manager then discussed what features to include in the remote. The group discussed the possibility of adding a shortcut function to the remote, and discussed the option of using a scroll wheel. The marketing expert discussed the options for the remote and the possibility to include a scrollwheel. The industrial designer discussed the internal workings of the remote with the team. The user interface designer discussed how to incorporate a scroll button into the remote design. The Project Manager discussed the requirements of the project and the team decided to include the scroll wheel in the design. '

In [ ]:
preds

NameError: ignored

In [ ]:
#predictions = []
#for i,txt in enumerate(test['input_text']):
#  predictions.append(model.predict(txt))
#  print(i)

In [ ]:
#dfpred = test['target_text']
#dfpred['pred'] = preds
#dfpred

In [ ]:
from rouge_metric import PyRouge

# Evaluate document-wise ROUGE scores
rouge = PyRouge(rouge_n=(1, 2, 3), rouge_l=True, rouge_w=True, rouge_s=True, rouge_su=True, skip_gap=4)
scores = rouge.evaluate(test["target_text"], pred['preds'])
print(scores)

{'rouge-1': {'r': 0.04879668417644621, 'p': 0.0002589082900096791, 'f': 0.0005150836193408556}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-3': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.04879668417644621, 'p': 0.0002589082900096791, 'f': 0.0005150836193408556}, 'rouge-w-1.2': {'r': 0.07908719484031634, 'p': 0.0004324366091242205, 'f': 0.0008601699413971202}, 'rouge-s4': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-su4': {'r': 0.0, 'p': 0.0, 'f': 0.0}}


In [ ]:
len(train['input_text'][0])

13676

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

# see ``examples/summarization/bart/run_eval.py`` for a longer example
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

ARTICLE_TO_SUMMARIZE = train['input_text'][0]

In [ ]:

inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1000, return_tensors='pt')

summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=1000, early_stopping=True)


In [ ]:

p = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

In [ ]:
p

["User Interface is a weekly, offbeat look at what's going on in the world of technology. This week, we look at a new remote control. We'll also hear from our marketing expert about the price of the device. And we'll also look at some of the features that make up the remote."]

In [ ]:
### NEW TRY  !!!!!!!!!!!!!!!

In [ ]:
# import modules and settings
! pip install torch==1.5.0 torchvision==0.6.0 -f https://download.pytorch.org/whl/torch_stable.html
! pip install transformers
!pip install rouge_metric
! pip install py-rouge
import pandas as pd
import numpy as np
import glob
import nltk
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
nltk.download('punkt')
pd.options.display.max_colwidth =1000


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
dfinput = pd.read_csv('/content/input_text.txt', sep=';')
abstractive = pd.read_csv('/content/abstr_text.txt', sep=';')
extractive = pd.read_csv('/content/extractive_text.txt', sep=';')

dfinput = dfinput.replace(np.nan, '', regex=True)


In [ ]:
for i,text in enumerate(dfinput['input_text']):
  text = text.replace('Oh.', '')
  text = text.replace('Oh', '')
  text = text.replace('Uh', '')
  text = text.replace('Uh.', '')
  text = text.replace('Hmm.', '')
  text = text.replace('Mm-hmm.', '')
  text = text.replace('Mm', '')
  text = text.replace('Mm.', '')
  text = text.replace('Um', '')
  text = text.replace('Um.', '')
  text = text.replace('-huh.', '')
  text = text.replace('..', '.')
  text = text.replace('. .', '.')
  text = text.replace('um', '.')
  text = text.replace('uh', '.')
  text = text.replace(',', '')
  text = text.replace("'", "")
  text = text.replace('"', "")
  dfinput.at[i, 'clean_text'] = text


In [ ]:

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
#model = model.to('cuda')
#tokenizer = tokenizer.to('cuda')

In [ ]:
# generate chunks of text \ sentences <= 1024 tokens
def nest_sentences(document):
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence)
        if length < 1024:
            sent.append(sentence)
        else:
            nested.append(sent)
            sent = [sentence]
            length = len(sentence)
    
    if sent:
        nested.append(sent)
    return nested


In [ ]:
# generate summary on text with <= 1024 tokens
def generate_summary(nested_sentences):
    summaries = []
    for nested in nested_sentences:
        input_tokenized = tokenizer.encode(' '.join(nested),truncation=False, return_tensors='pt')
        
        summary_ids = model.generate(input_tokenized,
                                      min_length=0,
                                      max_length=100
                                    )
        
        output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
        summaries.append(output)
    summaries = [sentence for sublist in summaries for sentence in sublist]
    return summaries

In [ ]:
nested = []
for i,text in enumerate(dfinput['clean_text']):
  nested.append(nest_sentences(text))

  
dfinput['nested'] = None
dfinput['nested'] = nested

In [ ]:
dfinput['nested'][48][0]

['Hi Im David and Im supposed to be an industrial designer.',
 'I just got the project announcement about what the project is.',
 'Designing a remote control.',
 'Thats about it didnt get anything else.',
 'Did you get the same thing?',
 'Cool.',
 'Theres too much gear.',
 'Okay.',
 'Cant draw.',
 'Yeah.',
 'well anyway I dont know its just the first animal I can think off the top of my head.',
 'Yes.',
 'Big reason is cause Im allergic to most animals.',
 'Allergic to animal fur so .',
 'fish was a natural choice.',
 'yeah and I kind of like whales.',
 'They come in and go eat everything in sight.',
 'And theyre quite harmless and mild and interesting.',
 'Tails a bit big I think.',
 'Its an after dinner dog then.',
 'It does make sense from maybe the design point of view cause you have more complicated characters like European languages then you need more buttons.',
 'So possibly.',
 'Yeah.',
 'And you keep losing them.',
 'Finding them is really a pain you know.',
 'I mean its usual

In [ ]:
#dfinput['nest_clean'] = []
nest_clean = []
for i, text in enumerate(dfinput['nested'][48][0]):
    tokens = text.split()
    token_cnt = len(tokens)
    char_cnt = sum([len(token) for token in tokens])
    if (token_cnt > 1):
      nest_clean.append(text)

      #dfinput.at([i, 'nest_clean']) = text
    #print("token count = %d, character count = %d," %(token_cnt, char_cnt))

In [ ]:
nest_clean

['Hi Im David and Im supposed to be an industrial designer.',
 'I just got the project announcement about what the project is.',
 'Designing a remote control.',
 'Thats about it didnt get anything else.',
 'Did you get the same thing?',
 'Theres too much gear.',
 'Cant draw.',
 'well anyway I dont know its just the first animal I can think off the top of my head.',
 'Big reason is cause Im allergic to most animals.',
 'Allergic to animal fur so .',
 'fish was a natural choice.',
 'yeah and I kind of like whales.',
 'They come in and go eat everything in sight.',
 'And theyre quite harmless and mild and interesting.',
 'Tails a bit big I think.',
 'Its an after dinner dog then.',
 'It does make sense from maybe the design point of view cause you have more complicated characters like European languages then you need more buttons.',
 'So possibly.',
 'And you keep losing them.',
 'Finding them is really a pain you know.',
 'I mean its usually quite small or when you want it right it slipp

In [ ]:
for word in list(words):  # iterating on a copy since removing will mess things up
    if word in stopwords:
        words.remove(word)

In [ ]:
from datetime import datetime
start=datetime.now()

sums = []
for i,text in enumerate(dfinput['nested'][48:52]): 
    sums.append(generate_summary(text))

print(datetime.now()-start)


0:02:44.750495


In [ ]:
refs = dfinput['clean_text'][48:52]
refs.reset_index(drop=True, inplace=True)
len(refs[3])

5056

In [ ]:
for i in range(len(sums)):
  #single speaker
  abrev = []
  slen = len(str(sums[i]).split())
  reflen = len(refs[i].split())
  abrevs = round(100*slen/reflen,0)
  abrev.append(abrevs)
  print('Summary length ',i , " :",slen, "words \n",
        'Original length ',i," :", reflen, "words \n",
        'in % : ',abrevs, "\n")

print('median text summarization per speaker : ',np.median(abrev), "%")  
sum = sums[0] + sums[1] + sums[2] + sums[3]
ref = refs[0] + refs[1] + refs[2] + refs[3]
suml= (len(str(sum).split()))
refl= (len(str(ref).split()))
print('total text summarization :', 100*round(suml/refl, 1), "%")

Summary length  0  : 66 words 
 Original length  0  : 224 words 
 in % :  29.0 

Summary length  1  : 304 words 
 Original length  1  : 1245 words 
 in % :  24.0 

Summary length  2  : 36 words 
 Original length  2  : 80 words 
 in % :  45.0 

Summary length  3  : 153 words 
 Original length  3  : 959 words 
 in % :  16.0 

median text summarization per speaker :  16.0 %
total text summarization : 20.0 %


In [ ]:
dfinput[48:52]

,id,speaker,input_text
48,ES2002a,A,"Hi, I'm David and I'm supposed to be an industrial designer. Um, I just got the project announcement about what the project is. Designing a remote control. That's about it, didn't get anything else. Did you get the same thing? Cool. There's too much gear. Okay. Can't draw. Um. Yeah. Um, well anyway, I don't know, it's just the first animal I can think off the top of my head. Um. Yes. Big reason is 'cause I'm allergic to most animals. Allergic to animal fur, so um fish was a natural choice. Um, yeah, and I kind of like whales. They come in and go eat everything in sight. And they're quite harmless and mild and interesting. Tail's a bit big, I think. It's an after dinner dog then. Hmm. It does make sense from maybe the design point of view 'cause you have more complicated characters like European languages, then you need more buttons. So, possibly. Hmm. Yeah. And you keep losing them. Finding them is really a pain, you know. I mean it's usually quite small, or when you want it right,..."
49,ES2002a,B,"Okay. Right. Um well this is the kick-off meeting for our our project. Um and um this is just what we're gonna be doing over the next twenty five minutes. Um so first of all, just to kind of make sure that we all know each other, I'm Laura and I'm the project manager. Do you want to introduce yourself again? Okay. Great. Okay. Um so we're designing a new remote control and um Oh I have to record who's here actually. So that's David, Andrew and Craig, isn't it? And you all arrived on time. Um yeah so des uh design a new remote control. Um, as you can see it's supposed to be original, trendy and user friendly. Um so that's kind of our our brief, as it were. Um and so there are three different stages to the design. Um I'm not really sure what what you guys have already received um in your emails. What did you get? Mm-hmm. Is that what everybody got? Okay. Um. So we're gonna have like individual work and then a meeting about it. And repeat that process three times. Um and at this point..."
50,ES2002a,C,"Um I'm Craig and I'm User Interface. Yeah. Well, my favourite animal would be a monkey. Then they're small cute and furry, and uh when planet of the apes becomes real, I'm gonna be up there with them. Yeah. I know um My parents went out and bought um remote controls because um they got fed up of having four or five different remote controls for each things the house. So um for them it was just how many devices control. Uh."
51,ES2002a,D,"Mm-hmm. Great. And I'm Andrew and I'm uh our marketing expert. Mm-hmm. Mm-hmm. Yeah, that's that's it. Yeah. I will go. That's fine. Alright. So This one here, right? Okay. Very nice. Alright. My favourite animal is like A beagle. Um charac favourite characteristics of it? Is that right? Uh, right, well basically um high priority for any animal for me is that they be willing to take a lot of physical affection from their family. And, yeah that they have lots of personality and uh be fit and in robust good health. So this is blue. Blue beagle. My family's beagle. I coulda told you a whole lot more about beagles. Boy, let me tell you. Impressionist. Alright. Mm. Superb sketch, by the way. Yep. I see a dog in there. Yep. Now I see a rooster. What kind is it? Is he aware that th it's his own cha tail he's chasing? Hmm. Probably when he was little he got lots of attention for doing it and has forever been conditioned. 'Kay. Um, can we just go over that again? Uh, so bas at twel Alright,..."


['Okay. well this is the kick-off meeting for our our project. and . this is just what were gonna be doing over the next twenty five minutes. so first of all just to kind of make sure that we all know each other Im Laura and Im the project manager. Okay. so were designing a new remote control and . I have to record whos here actually.',
 'you get to draw your favourite animal and s. up your favourite characteristics of it. So who would like to go first?',
 'And he does a funny thing where he chases his tail as well which is quite amusing so It is. He only does it after hes had his dinner and . hell just all of a sudden just get up and start chasing his tail round the living room. Yeah so . Yeah maybe.',
 'Think it will? . yeah regions and stuff yeah. Well for a remote control do you think that will be I suppose its depends on how complicated our remote control is.',
 'Yeah yeah. Five minutes to end of meeting. Were a bit behind. Right so do you think that should be like a main design a

In [ ]:
# example
sum = (''.join(sum))
sum

'Im allergic to animal fur so fish was a natural choice. I kind of like whales. They come in and go eat everything in sight. And theyre quite harmless and mild and interesting.I think one factor would be production cost. Because theres a cap there so . depends on how much you can cram into that price. I think that thats the main factor. Cool.Okay. well this is the kick-off meeting for our our project. and . this is just what were gonna be doing over the next twenty five minutes. so first of all just to kind of make sure that we all know each other Im Laura and Im the project manager. Okay. so were designing a new remote control and . I have to record whos here actually.you get to draw your favourite animal and s. up your favourite characteristics of it. So who would like to go first?And he does a funny thing where he chases his tail as well which is quite amusing so It is. He only does it after hes had his dinner and . hell just all of a sudden just get up and start chasing his tail ro

In [ ]:
extr_sum = extractive.target_text[extractive['ID']=='ES2002a']
extr_sum = "".join(extr_sum)
extr_sum

"Um well this is the kick-off meeting for our our project . so we're designing a new remote control and um Um , as you can see it's supposed to be original , trendy and user friendly . Um and so there are three different stages to the design . So we're gonna have like individual work and then a meeting about it . And repeat that process three times . So uh you get to draw your favourite animal and sum up your favourite characteristics of it . My favourite animal is like A beagle . Uh , right , well basically um high priority for any animal for me is that they be willing to take a lot of physical affection from their family . And , yeah that they have lots of personality Well Then they're small cute and furry , and I kind of like whales . They come in and go eat everything in sight . M my favourite animal is my own dog at home . Um he's very friendly and cheery and always pleased to see you , Um so according to the brief um we're gonna be selling this remote control for twenty five Euro

In [ ]:
abstr_sum = abstractive.target_text[abstractive['ID']=='ES2002a']
abstr_sum = "".join(abstr_sum)
abstr_sum

'The project manager introduced the upcoming project to the team members and then the team members participated in an exercise in which they drew their favorite animal and discussed what they liked about the animal. The project manager talked about the project finances and selling prices. The team then discussed various features to consider in making the remote. '

In [ ]:
 print(len(sum.split()) ,len(extr_sum.split()),len(abstr_sum.split())) # maxlen = 100:685 , max len = 200: 559 ; seqlen 512 : 1160 
 

559 868 58


In [ ]:
max_w = max(len(sum.split()) ,len(ref.split()))
import rouge

def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)


for aggregator in [ 'Avg', 'Individual']:
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=2,
                           limit_length=True,
                           length_limit=max_w,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

    hypothesis_1 = sum
    references_1 = [extr_sum, abstr_sum]


    all_hypothesis = [hypothesis_1]
    all_references = [references_1]

    scores = evaluator.get_scores(all_hypothesis, all_references)

    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
            for hypothesis_id, results_per_ref in enumerate(results):
                nb_references = len(results_per_ref['p'])
                for reference_id in range(nb_references):
                    print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                    print('\t' + prepare_results(results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
            print()
        else:
            print(prepare_results(results['p'], results['r'], results['f']))
    print()

Evaluation with Avg
	rouge-1:	P: 41.72	R: 56.08	F1: 47.85
	rouge-2:	P: 22.91	R: 30.80	F1: 26.27
	rouge-l:	P: 24.05	R: 30.77	F1: 27.00
	rouge-w:	P: 11.52	R:  3.80	F1:  5.71

Evaluation with Individual
	Hypothesis #0 & Reference #0: 
		rouge-1:	P: 77.30	R: 55.65	F1: 64.71
	Hypothesis #0 & Reference #1: 
		rouge-1:	P:  6.14	R: 62.07	F1: 11.18

	Hypothesis #0 & Reference #0: 
		rouge-2:	P: 45.30	R: 32.60	F1: 37.91
	Hypothesis #0 & Reference #1: 
		rouge-2:	P:  0.51	R:  5.26	F1:  0.93

	Hypothesis #0 & Reference #0: 
		rouge-l:	P: 39.63	R: 30.14	F1: 34.24
	Hypothesis #0 & Reference #1: 
		rouge-l:	P:  5.74	R: 39.46	F1: 10.02

	Hypothesis #0 & Reference #0: 
		rouge-w:	P: 19.38	R:  3.65	F1:  6.14
	Hypothesis #0 & Reference #1: 
		rouge-w:	P:  2.16	R:  9.70	F1:  3.54




In [ ]:
sumdic = { 'A':''.join(sums[0]), 'B':''.join(sums[1]), 'C':''.join(sums[2]), 'D':''.join(sums[3])}
sumdic



{'A': "David is an industrial designer. His project is to design a remote control. He is allergic to most animals.I think one factor would be production cost. Because there's a cap there, so . depends on how much you can cram into that price. I think that that's the main factor.",
 'B': "Laura: We're designing a new remote control and . I have to record who's here actually. So we're gonna have like individual work and then a meeting about it. And repeat that process three times.you get to draw your favourite animal and s. up your favourite characteristics of it. So who would like to go first?and . And he does a funny thing where he chases his tail, which is quite amusing, so It is. He only does it after he's had his dinner and . he'll just all of a sudden just get up and start chasing his tail 'round the living room.But I I don't know, I mean do you think the fact that it's going to be sold internationally will have a bearing on how we design it at all? Think it will? . yeah, regions a

In [ ]:
sumdf = pd.DataFrame.from_dict(sumdic,columns=['summary'], orient='index')
sumdf.to_csv(r'/content/summ.txt',sep=' ',mode= 'a' ,index=True, header=True)

In [ ]:


with open('file.txt', 'w') as file:
     file.write(str(sumdic)) # use `json.loads` to do the reverse